In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
import torch.nn.functional as F


In [3]:
#Text-CNN parameter
keral_sizes=3
num_keral=3
stride=1
input_length=3
embedding_size=3
#生成word2id and vocab词典 
sentences=['i love you','he loves me','she likes baseball','i hate you','sorry for that','this is awful']
labels=[0,0,0,1,1,1]
vocab=' '.join(sentences).split()
vocab=list(set(vocab))
word2id={word:index for index,word in enumerate(vocab)}
vocab_size=len(vocab)
def construct_tensor(input_x):
    x=[]
    for sen in input_x:
        x.append([word2id[word] for word in sen.split()])#这种语法方式产生的列表迭代对象，是一个列表生成器
    x=list(x)
    input_numpy=np.asarray(x,dtype=np.int64)#当为索引的时候需要将类型变为int 64位或者在numpy转变为tensor时直接用LongTensor()
    print(input_numpy)
    input_tensor=torch.from_numpy(input_numpy)
    return input_tensor
input_tensor=construct_tensor(sentences)
label_tensor=torch.tensor(np.asarray(labels,dtype=np.int64))
#input_tensor=torch.LongTensor(input_tensor)
#print(input_tensor)
#print(output_tensor)
#二维生成三维的方式利用将二维的tensor嵌入二维的tensor会出现三维tensor

#print(w[input_tensor])

[[15 11  7]
 [ 9 13 10]
 [ 2  6  1]
 [15  5  7]
 [ 0 14  4]
 [12  8  3]]


In [4]:
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN,self).__init__()
        self.W=nn.Parameter(torch.empty(vocab_size,embedding_size).uniform_(-1,1))
        self.conv=nn.Conv2d(1,3,3,stride=1)
        self.relu=nn.ReLU()
        self.max_pool=nn.MaxPool2d(1,stride=1)
        self.linear=nn.Linear(3,2)
        self.softmax=nn.LogSoftmax(1)
    def forward(self,x):
        embedding=self.W[x]
        embedding=embedding.unsqueeze(1)
        output=self.conv(embedding)
        output=self.relu(output)
        output=self.max_pool(output)
        #output=torch.cat(output,1)
        output=torch.reshape(output,[-1,3])
        output=self.linear(output)
        output=self.softmax(output)
        return output
        

        
            
            
                
      

In [5]:
#traing定义超参数
#iteration=len(labels)/batch_size#遍历一次数据集需要进行的迭代次数
epoch=5000#遍历数据集的次数
criterion=nn.NLLLoss()
model=TextCNN()
optimizer=optim.Adam(model.parameters(),lr=0.001)
print(model)

TextCNN(
  (conv): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
  (relu): ReLU()
  (max_pool): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (linear): Linear(in_features=3, out_features=2, bias=True)
  (softmax): LogSoftmax()
)


In [6]:
for i in range(epoch):
    output=model(input_tensor)
    loss=criterion(output,label_tensor)
    if (i+1)%1000 ==0:
        print('Epoch: %04d'%(i+1),'cost=','{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()
  

Epoch: 1000 cost= 0.000000
Epoch: 2000 cost= 0.000000
Epoch: 3000 cost= 0.000000
Epoch: 4000 cost= 0.000000
Epoch: 5000 cost= 0.000000


In [14]:
test_text=['sorry hate you']
test_tensor=construct_tensor(test_text)
predict=model(test_tensor)
#print(predict)
predict=predict.max(1,keepdim=True)
print(predict[1][0].item())
if predict[1][0]==0:
    print(test_text,'is Bad Mean...')
else:
    print(test_text,'is Good Mean...')

[[0 5 7]]
1
['sorry hate you'] is Good Mean...


In [ ]:
print(list(model.parameters()))

In [ ]:
torch.save(model.state_dict(),'f:\model\cnn.pkl')

In [ ]:
print(model.state_dict())

In [ ]:
'''
总结：
(1)利用二维词向量权重矩阵以及二维数据产生三维的tensor数据集合
（2）[内的操作产生的为列表生成器，所以效率高]
（3）epoch 进行几次迭代，batch-size 一批的大小，iteration也叫training step，每次迭代一次更新一次网络结构参数。iteration=n/batch-size

'''